# PENGENALAN GAN
Rizky Ramdhani Koswara - 11122300 - 4KA25
> Add blockquote



In [ ]:
from keras.datasets import mnist

Penjelasan: Blok ini hanya berisi satu baris kode yang berfungsi untuk mengimpor dataset MNIST dari library Keras. Dataset MNIST adalah kumpulan data populer yang berisi 70.000 gambar hitam-putih tulisan tangan angka dari 0 hingga 9. Dataset ini sering digunakan sebagai "Hello, World!" dalam dunia machine learning untuk tugas pengenalan gambar.

In [ ]:
from tensorflow.keras import Sequential
from keras.layers import BatchNormalization, Dense, Reshape, Flatten
from keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np

Penjelasan: Blok ini mengimpor semua komponen yang dibutuhkan untuk membangun model jaringan saraf tiruan (neural network) dengan Keras.

Sequential: Ini adalah model dasar di Keras yang memungkinkan kita membangun jaringan saraf secara berurutan, lapisan demi lapisan.

Dense, Reshape, Flatten: Ini adalah jenis-jenis lapisan (layer) yang akan digunakan.

Dense: Lapisan yang terhubung sepenuhnya (fully connected), di mana setiap neuron terhubung ke semua neuron di lapisan sebelumnya.

Flatten: Mengubah data multi-dimensi (seperti gambar 28x28) menjadi satu dimensi (vektor).

Reshape: Mengubah bentuk (dimensi) data ke bentuk yang diinginkan.

BatchNormalization: Sebuah teknik untuk menstabilkan dan mempercepat proses pelatihan dengan menormalisasi input dari setiap lapisan.

LeakyReLU: Fungsi aktivasi yang merupakan varian dari ReLU. Ini sering digunakan pada GAN karena membantu mengatasi masalah "dying ReLU" dan menjaga aliran gradien selama pelatihan.

Adam: Salah satu algoritma optimisasi yang paling populer untuk melatih model deep learning.

numpy: Library standar untuk komputasi numerik di Python, digunakan di sini untuk operasi matematika pada array.

In [ ]:
## mendefinisikan variable gambar
## ukuran dimensi
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0001)

Penjelasan: Blok ini mendefinisikan variabel-variabel penting yang akan digunakan di seluruh program.

img_width, img_height, channels: Menentukan dimensi gambar dari dataset MNIST (lebar 28 piksel, tinggi 28 piksel, dan 1 channel warna karena gambarnya hitam-putih).

img_shape: Menggabungkan dimensi tersebut menjadi satu tuple (28, 28, 1), yang akan digunakan untuk mendefinisikan bentuk input dan output model.

latent_dim: Menentukan ukuran "ruang laten" atau dimensi dari vektor input acak (noise) untuk Generator. Angka 100 berarti Generator akan membuat gambar dari input acak yang terdiri dari 100 angka.

adam: Menginisialisasi optimizer Adam dengan learning_rate (laju belajar) yang diatur cukup kecil (0.0001) untuk membantu pelatihan GAN agar lebih stabil.

In [ ]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.summary()
  return model

In [ ]:
generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
Total params: 26880 (105.00 KB)
Trainable params: 26368 (103.00 KB)
Non-trainable params: 512 (2.00 KB)
_________________________________________________________________


In [ ]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model

In [ ]:
generator = build_generator()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_2 (Bat  (None, 512)               2048      
 chNormalization)                                     

Penjelasan: Blok ini mendefinisikan arsitektur untuk Generator. Tugas Generator adalah mengambil input acak (noise) berdimensi latent_dim (100) dan mengubahnya menjadi sebuah gambar berukuran 28x28x1.

Model ini secara bertahap meningkatkan dimensi dari input 100 menjadi 256, lalu 512, dan 1024 melalui lapisan Dense. Setiap lapisan Dense diikuti oleh LeakyReLU dan BatchNormalization untuk menjaga stabilitas.

Lapisan Dense terakhir menghasilkan output sejumlah total piksel pada gambar (28*28*1 = 784) dengan fungsi aktivasi tanh. tanh digunakan agar nilai piksel output berada di rentang [-1, 1], yang merupakan praktik umum dalam GAN.

Lapisan Reshape terakhir mengubah vektor 784 piksel tersebut menjadi bentuk gambar yang sesuai, yaitu (28, 28, 1).

model.summary() mencetak ringkasan arsitektur model yang telah dibuat.

In [ ]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))

  model.summary()
  return model

discriminator = build_discriminator()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_5 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 256)               0         
                                                                 
Total params: 533248 (2.03 MB)
Trainable params: 533248 (2.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Penjelasan: Blok ini mendefinisikan arsitektur untuk Diskriminator. Tugas Diskriminator adalah menerima sebuah gambar (baik asli dari dataset atau palsu dari Generator) dan memutuskan apakah gambar itu asli atau palsu.

Lapisan Flatten pertama kali meratakan gambar input (28, 28, 1) menjadi vektor satu dimensi (784 piksel).

Lapisan Dense selanjutnya (512 dan 256 neuron) berfungsi sebagai "otak" dari diskriminator untuk mengekstraksi fitur dari gambar dan melakukan klasifikasi.

Penting: Arsitektur ini belum lengkap. Seharusnya ada lapisan output terakhir seperti Dense(1, activation='sigmoid') untuk menghasilkan satu nilai probabilitas (antara 0 untuk "palsu" dan 1 untuk "asli").

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer='adam')

Penjelasan: Ini adalah blok yang sangat penting di mana model Generator dan Diskriminator digabungkan untuk membentuk model GAN utuh.

discriminator.compile(...): Pertama, Diskriminator dikompilasi sebagai model mandiri. Ia menggunakan binary_crossentropy sebagai fungsi kerugian, karena tugasnya adalah klasifikasi biner (asli vs. palsu).

GAN = Sequential(): Sebuah model Sequential baru dibuat untuk menampung keseluruhan arsitektur GAN.

discriminator.trainable = False: Ini adalah trik kunci dalam pelatihan GAN. Ketika kita melatih Generator, kita tidak ingin bobot Diskriminator ikut berubah. Dengan mengatur ini ke False, kita "membekukan" Diskriminator. Tujuannya adalah agar Generator bisa belajar dari feedback Diskriminator tanpa mengubah Diskriminator itu sendiri.

GAN.add(generator) dan GAN.add(discriminator): Generator dan Diskriminator (yang sudah dibekukan) ditambahkan ke model GAN secara berurutan. Ini berarti output dari Generator akan langsung menjadi input bagi Diskriminator.

GAN.compile(...): Model GAN gabungan ini kemudian dikompilasi. Tujuannya adalah untuk melatih Generator agar menghasilkan gambar yang bisa menipu Diskriminator sehingga Diskriminator mengklasifikasikannya sebagai "asli".